In [1]:
import pandas as pd
import pathlib
import matplotlib.pyplot as plt

In [2]:
interim = pathlib.Path().cwd().parent.parent / 'data' / 'interim'

In [3]:
df = (pd.read_csv(interim / 'public-test.csv', sep=';')
      [['date_report', 'sex', 'yob', 'addr_dist_home', 'name_full']]
     )

/home/kyo/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (1,8,9,10,11,13,19,20,23,25,26,27,28,29,30,33,34,35,36,39,40,41,44,49,50,51,52,55,56,57,58,61,62,63,64,67,68,69,70,73,74,75,78,81,82,83,84,85,86,87,88,89,90,91,92,95,96,99,101,105,108) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df = df.assign(
    date_report = pd.to_datetime(df.date_report, errors='coerce', format='%d/%m/%Y'),
    adult = (2021 - df.yob.astype('float')) > 18
)

In [5]:
df = df[(df.sex == 'NAM') | (df.sex == 'NỮ')]
df = df.replace(['NAM', 'NỮ'], ['MALE', 'FEMALE'])

In [21]:
# inc

In [7]:
inc = (
    df[['date_report']]
    .groupby(['date_report'])
    .apply(lambda x: len(x))
    .to_frame(name='case')
    .reindex(pd.date_range(start=df.date_report.min(), end=df.date_report.max(), freq='D'))
    .fillna(0)
      )
inc['case_rm'] = inc.case.rolling(7).mean()
# inc['case_rs'] = inc.case.rolling(7).sum()

# inc['case_rs'] = inc.rolling(7).sum()

In [10]:
def calculate_incidence(df, group_col, roll_mean=False, pivot=True):
    inc = (
        df[['date_report', group_col]]
        .groupby(['date_report', group_col])
        .apply(lambda x: len(x))
        .to_frame(name='case')
        .unstack(fill_value=0)
        .asfreq('D', fill_value=0)
        .stack()
        .sort_index(level=0)
        .reset_index(1)
    )
    
    if roll_mean:
        inc['case_rm'] = (
            inc.groupby(group_col)['case']
            .transform(lambda x: x.rolling(7).mean())
        )

    inc = inc.reset_index(0)
    
    if pivot:
        inc = inc.pivot(index='date_report', columns=group_col, values=['case'])
#         col = inc[group_col].unique()
#         inc.columns = col
        
    return inc

In [19]:
inc_adh_rollmean = calculate_incidence(df, 'addr_dist_home', roll_mean=True, pivot=False)
inc_adh_cumsum = (
    calculate_incidence(df, 'addr_dist_home', roll_mean=False, pivot=False)
    .groupby(['addr_dist_home', 'date_report'])
    .sum()
    .groupby(level=0)
    .cumsum()
    .reset_index()
)
inc_sex = calculate_incidence(df, 'sex', roll_mean=False, pivot=True)
inc_sex.columns = ['female', 'male']
inc_adult = calculate_incidence(df, 'adult', roll_mean=False, pivot=True)
inc_adult.columns = ['lte18', 'gt18']

In [23]:
# inc_adh_cumsum[inc_adh_cumsum.addr_dist_home == 'QA08']
# inc_adult

In [ ]:
inc_sex = (df[['date_report', 'sex']]
           [(df.sex == 'NAM') | (df.sex == 'NỮ')]
           .groupby(['date_report', 'sex'])
           .apply(lambda x: len(x))
           .to_frame(name='case')
          )

inc_sex = inc_sex.reset_index()
inc_sex = inc_sex.pivot(index='date_report', columns='sex', values='case')
inc_sex.columns = ['male', 'female']

In [ ]:
inc_adult = (df[['date_report', 'adult']]
           .groupby(['date_report', 'adult'])
           .apply(lambda x: len(x))
           .to_frame(name='case')
          )

inc_adult = inc_adult.reset_index()
inc_adult = inc_adult.pivot(index='date_report', columns='adult', values='case')
inc_adult.columns = ['gt18', 'lte18']

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
# ax.bar(inc.index, inc.case, label='Số ca mới theo ngày', linewidth=3, color='gray')
ax.bar(inc_sex.index, inc_sex.male, label='Nam', color='blue', alpha=0.4)
ax.bar(inc_sex.index, inc_sex.female, bottom=inc_sex.male, label='Nữ', color='red', alpha=0.4)
ax.plot(inc.index, inc.case_rm, label='Biến động trung bình 7 ngày', linewidth=3, color='black')
# ax.plot(inc_sex.index, inc_sex.male, label='Nam', linewidth=2, color='blue')
# ax.plot(inc_sex.index, inc_sex.female, label='Nữ', linewidth=2, color='red')
ax.set_ylabel('Số ca mới')
ax.set_xlabel('Ngày')
ax.set_xticks(inc.index)
ax.tick_params(axis='x', labelrotation=90)
ax.set_title('Số ca mới theo ngày và biến động trung bình 7 ngày')
ax.legend()

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
ax.bar(inc.index, inc.case, label='Số ca mới theo ngày', linewidth=3, color='gray')
ax.plot(inc.index, inc.case_rm, label='Biến động trung bình 7 ngày', linewidth=3, color='black')
ax.plot(inc_adult.index, inc_adult.gt18, label='> 18', linewidth=2, color='blue')
ax.plot(inc_adult.index, inc_adult.lte18, label='<= 18', linewidth=2, color='red')
ax.set_ylabel('Số ca mới')
ax.set_xlabel('Ngày')
ax.set_xticks(inc.index)
ax.tick_params(axis='x', labelrotation=90)
ax.set_title('Số ca mới theo ngày và biến động trung bình 7 ngày')
ax.legend()

In [ ]:
inc